# Adding user-defined influence, reaction model and GUI: The segregation model

The segregation model has been proposed by [Thomas Schelling](https://en.wikipedia.org/wiki/Thomas_Schelling) in 1971 in his famous paper [Dynamic Models of Segregation](https://www.stat.berkeley.edu/~aldous/157/Papers/Schelling_Seg_Models.pdf). The goal of this model is to show that segregation can occur even if it is not wanted by the agents.

In our implementation of this model, turtles are located in the grid and at each step, compute an happiness index based on the similarity of other agents in their neighborhood. If this index is below a value, called here similarity rate, the turtle wants to move to an other location.

## Grab dependencies

In [1]:
#!python
#@repository("~/.m2/repository/")
#@dependency(group="fr.univ-artois.lgi2a", module="similar2logo-lib", version="1.0-SNAPSHOT")

import math

from fr.univ_artois.lgi2a.similar.extendedkernel.levels import ExtendedLevel
from fr.univ_artois.lgi2a.similar.extendedkernel.libs.abstractimpl import AbstractAgtDecisionModel
from fr.univ_artois.lgi2a.similar.extendedkernel.libs.timemodel import PeriodicTimeModel
from fr.univ_artois.lgi2a.similar.microkernel import AgentCategory
from fr.univ_artois.lgi2a.similar.microkernel.ISimulationModel import AgentInitializationData
from fr.univ_artois.lgi2a.similar.microkernel.influences import RegularInfluence
from fr.univ_artois.lgi2a.similar2logo.kernel.initializations import AbstractLogoSimulationModel
from fr.univ_artois.lgi2a.similar2logo.kernel.model import LogoSimulationParameters
from fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle import TurtleFactory, \
    TurtleAgentCategory
from fr.univ_artois.lgi2a.similar2logo.kernel.model.levels import LogoSimulationLevelList, \
    LogoDefaultReactionModel
from fr.univ_artois.lgi2a.similar2logo.lib.model import ConeBasedPerceptionModel
from fr.univ_artois.lgi2a.similar2logo.lib.tools.html import Similar2LogoHtmlRunner
from fr.univ_artois.lgi2a.similar.extendedkernel.libs.random import PRNG
from java.awt.geom import Point2D
from java.util import ArrayList

Resolving dependency: fr.univ_artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT {default=[default]}
Preparing to download artifact fr.univ_artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT!similar2logo-lib.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar2logo-kernel;1.0-SNAPSHOT!similar2logo-kernel.jar
Preparing to download artifact com.sparkjava#spark-core;2.7.2!spark-core.jar(bundle)
Preparing to download artifact org.slf4j#slf4j-simple;1.7.13!slf4j-simple.jar
Preparing to download artifact org.apache.commons#commons-math3;3.6.1!commons-math3.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-microKernel;1.0-SNAPSHOT!similar-microKernel.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-microKernel-commonLibs;1.0-SNAPSHOT!similar-microKernel-commonLibs.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-extendedKernel;1.0-SNAPSHOT!similar-extendedKernel.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-extendedKernel-extendedLibs;1

No Outputs

## Model parameters

We define the following parameters and their default values.

In [2]:
class SegregationSimulationParameters(LogoSimulationParameters): 

    def __init__(self):
        self.similarityRate = 3.0/8
        self.vacancyRate = 0.05
        self.perceptionDistance = math.sqrt(2)

No Outputs

## Model-specific influence

We define an influence called `Move` that is emitted by an agent who wants to move to another location. It is defined by a  unique identifier, here "move", and the state of the turtle that wants to move.

In [3]:
class Move(RegularInfluence):
    
    CATEGORY = 'move'
    
    def __init__(self, timeLowerBound, timeUpperBound, target):
        self.target = target
        super(Move, self).__init__(
            self.CATEGORY,
            LogoSimulationLevelList.LOGO,
            timeLowerBound,
            timeUpperBound
        )

No Outputs

## Decision model

The decision model computes a happiness index based on the rate of turtles of different categories in its neighborhood. If the index is below the parameter `similarityRate`, the turtle emits a `Move` influence.

In [4]:
class SegregationDecisionModel(AbstractAgtDecisionModel):
    
    def __init__(self, parameters):
        self.parameters = parameters
        super(SegregationDecisionModel, self).__init__(
            LogoSimulationLevelList.LOGO
        )
        
    def decide(
        self,
        timeLowerBound,
        timeUpperBound,
        globalState,
        publicLocalState,
        privateLocalState,
        perceivedData,
        producedInfluences
    ):
        similarityRate = 0.0
        for perceivedTurtle in perceivedData.turtles:
            if perceivedTurtle.content.categoryOfAgent.isA(
               publicLocalState.categoryOfAgent
            ):
                similarityRate+=1

        if not perceivedData.turtles.isEmpty():
            similarityRate /= perceivedData.turtles.size()
        if similarityRate < self.parameters.similarityRate:
            producedInfluences.add(
                Move(timeLowerBound, timeUpperBound, publicLocalState)
            )

No Outputs

## Reaction model

The reaction model handles the `Move` influences emitted by unhappy turtles. First, it identifies vacant places and moves the turtles that have emitted a `Move` influence. Note that if there is not enough vacant places, not all turtle wishes can be fulfilled.

In [5]:
class SegregationReactionModel(LogoDefaultReactionModel):
        
        def __init__(self,parameters):
            self.parameters = parameters

        def makeRegularReaction(
            self,
            transitoryTimeMin,
            transitoryTimeMax,
            consistentState,
            regularInfluencesOftransitoryStateDynamics,
            remainingInfluences
        ):
            #If there there is at least an agent that wants to move
            if regularInfluencesOftransitoryStateDynamics.size() > 2:
                specificInfluences = ArrayList()
                vacantPlaces = ArrayList()
                specificInfluences.addAll(
                    regularInfluencesOftransitoryStateDynamics
                )
                PRNG.get().shuffle(specificInfluences)
                #Identify vacant places
                envState = consistentState.publicLocalStateOfEnvironment
                for x in range(0, envState.width):
                    for y in range(0,envState.height):
                        if envState.getTurtlesAt(x, y).isEmpty():
                            vacantPlaces.add(
                                Point2D.Double(x, y)
                            )
                PRNG.get().shuffle(vacantPlaces)
                #move agents
                i = 0
                for influence in specificInfluences:
                    if influence.category == Move.CATEGORY:
                        envState.turtlesInPatches[
                            int(math.floor(influence.target.location.x))
                        ][
                            int(math.floor(influence.target.location.y))
                        ].clear()
                        envState.turtlesInPatches[
                            int(math.floor(vacantPlaces[i].x))
                        ][
                            int(math.floor(vacantPlaces[i].y))
                        ].add(influence.target)
                        influence.target.location = vacantPlaces[i]
                        i+=1
                    if i >= vacantPlaces.size():
                        break

No Outputs

## Simulation model

The simulation model generates the Logo level using the user-defined reaction model and a simple periodic time model. It also generates turtles of 2 different types (a and b) randomly in the grid with respect to the vacancy rate parameter.

In [6]:
class SegregationSimulationModel(AbstractLogoSimulationModel):
    
    def __init__(self, parameters):
        super(SegregationSimulationModel, self).__init__(parameters)
        
    def generateLevels(self, simulationParameters):
        logo = ExtendedLevel(
            simulationParameters.initialTime,
            LogoSimulationLevelList.LOGO,
            PeriodicTimeModel(
                1,
                0,
                simulationParameters.initialTime
            ),
            SegregationReactionModel(simulationParameters)
        )
        levelList = []
        levelList.append(logo)
        return levelList
    
    def generateAgents(self, parameters, levels):
        result = AgentInitializationData()
        t = ''
        for x in range(0, parameters.gridWidth):
            for y in range(0, parameters.gridHeight):
                if PRNG.get().randomDouble() >= parameters.vacancyRate:
                    if PRNG.get().randomBoolean():
                        t = 'a'
                    else:
                        t = 'b'
                    turtle = TurtleFactory.generate(
                            ConeBasedPerceptionModel(
                                parameters.perceptionDistance,
                                2 * math.pi,
                                True,
                                False,
                                False
                            ),
                            SegregationDecisionModel(parameters),
                            AgentCategory(t, [TurtleAgentCategory.CATEGORY]),
                            0.0,
                            0.0,
                            0.0,
                            x,
                            y
                    )
                    result.agents.add(turtle)
        return result

No Outputs

## Launch the HTML runner

The GUI is defined in a variable called `segregationgui`. Finally, we launch the web server with the above described GUI.

In [7]:
segregationgui = '''
    <canvas id='grid_canvas' class='center-block' width='400' height='400'></canvas>
    <script type='text/javascript'>
        drawCanvas = function (data) {
            var json = JSON.parse(data),
                canvas = document.getElementById('grid_canvas'),
                context = canvas.getContext('2d');
            context.clearRect(0, 0, canvas.width, canvas.height);
            for (var i = 0; i < json.agents.length; i++) {
                var centerX = json.agents[i].x * canvas.width;
                var centerY = json.agents[i].y * canvas.height;
                var radius = 2;
                if (json.agents[i].t == 'a') {
                    context.fillStyle = 'red';
                } else {
                    context.fillStyle = 'blue';
                }
                context.beginPath();
                context.arc(centerX, centerY, radius, 0, 2 * Math.PI, false);
                context.fill();
            }
        }
    </script>'''

runner = Similar2LogoHtmlRunner()
runner.config.setExportAgents(True)
model = SegregationSimulationModel(SegregationSimulationParameters())
runner.config.setCustomHtmlBodyFromString(segregationgui)
runner.initializeRunner(model)
runner.showView()    

No Outputs